#Class Definitions

In [ ]:
class Group(object):
    """
        Grouping of Records
    """
    def __init__(self, EID_json=None):
        """
            Constructor
        """
        
        self.records = []
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
        def _parse_EID_json(self, mjson):
            """
                To be implemented
            """
            pass

In [ ]:
import iso8601

class Record(object): #RECORD
    """
        The Record object is designed to contain all metadata for a single record.
        It can be initialized by passing the EnergyID metadata json to the constructor,
        or by calling the parse_EID_json method.
        Parsing the JSON is optional and put in a seperate method so there is room
        to add other metadata formats.
    """
    def __init__(self, EID_json=None):
        """
            Object constructor. If the EnergyID JSON is passed, all metadata and
            datapoints are parsed and added. It is left optional though so other methods
            of initializing can be added in the future.
            
            Parameters
            ----------
            EID_json: JSON string from EnergyID containing all metadata for a single record
        """
        self.points = []
        if EID_json is not None:
            self._parse_EID_json(EID_json)
    
    def _parse_EID_json(self,mjson):
        """
            Method that parses the EnergyID JSON
            
            Parameters
            ----------
            mjson: JSON string from EnergyID containing all metadata for a single record
        """
        
        self.id = mjson['id']
        self.version = mjson['version']
        self.created = iso8601.parse_date(mjson['created']) #parse to datetime
        self.active = mjson['status'] == 'active' #parse to bool
        
        #We save the context and weatherstation metadata as raw JSON for now.
        self._context = mjson['context']
        self.weatherStations = mjson['weatherStations']
        
        #Data Points are objects. We create a list of objects here.
        for pjson in mjson['points']:
            self.points.append(Point(parent_record = self, EID_json = pjson))
            
    def get_property(self,tag):
        """
            Method that parses context metadata
            
            Parameters
            ----------
            tag: String, tag of the property to be parsed
            
            Returns
            -------
            Parsed value for the given tag.
            None if the tag was not found.
        """
        if tag in self._context:
            return self._context[tag]
        else:
            return None
    
    def get_points_by_metric(self, metric_name):
        """
            Lookup list of points for a metric_name
            
            Parameters
            ----------
            metric_name: String
            
            Returns
            -------
            List of Points
        """
        return [point for point in self.points if point.has_metric(metric_name)]

In [ ]:
class Point(object):
    """
        The Point object countains all metadata for a Data Point,
        and all coupled metrics.
    """
    def __init__(self, parent_record, EID_json=None):
        """
            Construction method. EnergyID JSON is optional so that
            a data point can be constructed by via another method in the future.
            
            Parameters
            ----------
            parent_record: Record object, parent to this Data Point
            EID_json: EnergyID JSON for this data Point (optional)
        """
        self.parent_record = parent_record
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self,mjson):
        """
            Parse the EnergyID JSON into metadata, metrics and dimensions
            
            Parameters
            ----------
            mjson: JSON String, EnergyID JSON for a single point
        """
        self.id = mjson['id']
        self.active = mjson['status'] == 'active' #parse to bool
        
        #save al other properties as raw json and parse when requested
        self._raw_json = mjson
        
        #create list of coupled metrics and dimensions
        self.metrics = [Metric(parent_point = self, EID_json = cjson) for cjson in mjson['metrics']]
        
    def get_propery(self,tag):
        """
            Method that parses context metadata
            
            Parameters
            ----------
            tag: String, tag of the property to be parsed
            
            Returns
            -------
            Parsed value for the given tag.
            None if the tag was not found.
        """
        if tag in self._raw_json:
            return self._raw_json[tag]
        else:
            return None
    
    def has_metric(self, metric_name):
        """
            Check if this point has a metric
            
            Parameters
            ----------
            metric_name: String
            
            Returns
            -------
            bool
        """
        for metric in self.metrics:
            if metric.name == metric_name:
                return True
        return False

In [ ]:
class Metric(object):
    """
        Object contains info for a metric
    """
    def __init__(self, parent_point, EID_json = None):
        """
            Constructor
            
            Parameters
            ----------
            parent_point: parent Point
            EID_json: string
            
        """
        self.parent_point = parent_point
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self, mjson):
        
        self.name = mjson['name']
        self.coefficient = mjson['coefficient']
        self.dimensions = [Dimension(parent_metric = self, EID_json = cjson) for cjson in mjson['dimensions']]

In [ ]:
class Dimension(object):
    """
        Object contains info for a dimension
    """
    def __init__(self, parent_metric, EID_json = None):
        """
            Constructor
            
            Parameters
            ----------
            parent_metric: parent
            EID_json: string
        """
        
        self.parent_metric = parent_metric
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self, mjson):
        self.name = mjson['name']
        self.value = mjson['value']

#Parsing

In [ ]:
import json

In [ ]:
with open('metadata_eid.json') as data_file:    
    mjson = json.load(data_file)

In [ ]:
d = Record(EID_json=mjson)

In [ ]:
print d.id, d.version, d.created, d.active

In [ ]:
d.get_property('postalCode'), d.get_property('dwellingType')

In [ ]:
for w in d.weatherStations:
    print w

In [ ]:
for p in d.points:
    print p.id
    print p.get_propery('displayName')
    print p.metrics
    print '\n'

In [ ]:
for p in d.points:
    for m in p.metrics:
        print m.name, m.coefficient
        for dim in m.dimensions:
            print "\t",
            print dim.name, dim.value
    print '\n'

In [ ]:
d.get_points_by_metric('ea:distanceTravelled')